In [3]:
from nltk.corpus import wordnet as wn
import json
from itertools import chain
import json

In [4]:
# Getting all words that we have in NLTK & Webster
allWords = set(chain(*[ss.lemma_names() for ss in wn.all_synsets()]))
print(len(allWords))
with open("dictionary_compact.json",'r') as f:
        data = json.load(f)
for i in data:
        allWords.add(i)        
len(allWords)

148730


212200

In [5]:
# Getting all jsons line by line from wiktionary English Data



lst = []
with open("kaikki.org-dictionary-English.json", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        lst.append(data)

In [6]:
def formatUsage(body):
    usageList = []
    definitions = []
    try:
        
        for i in body['senses']:
            definition = i['glosses'][0].split('\n') if 'glosses' in i else [""]
            definition = definition[0]
            if definition in definitions:
                continue
            definitions.append(definition)

            if 'examples' in i:
                for j in i['examples']:
                    j['source'] = 'Wiktionary'

            examples = i['examples'] if 'examples' in i else []

            usageList.append({
                'definition': {
                                'gloss':definition,
                                'source':'Wiktionary',
                },
                'examples': examples
            })
    except:
        return usageList
    
    return usageList

def formatAudio(body):
    audioList = []
    if 'sounds' in body:
        for j in body['sounds']:
            if ('mp3_url' in j):
                for i in audioList:
                    if 'tags' in j and  i['tags'] == j['tags'][0]:
                        continue
                audioList.append({
                    'audioLink': j['mp3_url'],
                    'tags' : j['tags'][0] if 'tags' in j else "",
                    'source': 'Wiktionary'

                })
    return audioList            
           




    

def formatObject(body):
    return {
        'pos'  : body['pos'],
        'etymology_text' : body['etymology_text'] if 'etymology_text' in body else "",
        'etymology_number' : body['etymology_number'] if 'etymology_number' in body else "",
        'definitions': formatUsage(body),
        'audio':formatAudio(body),
        'source':'Wiktionary'

    }
    

In [7]:
# Getting all objects/entries related to the word in payload
payload= {}
for i in lst:
    word = i['word']
    if word in allWords:
        payload[word]= [formatObject(i)] if word not in payload else payload[word]+[formatObject(i)]
print(len(payload))    

124686


In [8]:
finalProcessedWords = []
for i in payload:
    finalProcessedWords.append({
        'word':i,
        'usage': payload[i]
    })




In [11]:
json_object = json.dumps(finalProcessedWords, indent=4)
with open("websterWordNet.json", "w") as outfile:
    outfile.write(json_object)        


In [10]:
# Formatting payload for each word in allWords in the required format for our database
